<a href="https://colab.research.google.com/github/RC11-SkillsClass2022-23/XingFeng/blob/main/XingFengVideoScoreHomework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import PIL
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dropout, Dense, Softmax)
from tensorflow.keras.applications import mobilenet as _mobilenet
import random
import os
import numpy as np

In [2]:
!pip install --upgrade moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! git clone https://github.com/kenshohara/video-classification-3d-cnn-pytorch.git

fatal: destination path 'video-classification-3d-cnn-pytorch' already exists and is not an empty directory.


In [4]:
%cd video-classification-3d-cnn-pytorch/

/content/video-classification-3d-cnn-pytorch


In [5]:
import json
import os
from moviepy.editor import *
from moviepy import *

In [6]:
videos = os.listdir('/content/drive/MyDrive/SkillWorkshop/VideoProcess')

with open('input', 'w') as inputfile:
  for v in videos:
    if v.split('.')[-1] == 'mp4':
      inputfile.write(v+'\n')

In [7]:
! python main.py --input input --video_root /content/drive/MyDrive/SkillWorkshop/VideoProcess --output /content/drive/MyDrive/SkillWorkshop/output.json --model /content/drive/MyDrive/SkillWorkshop/resnet-34-kinetics-cpu.pth --model_depth 34 --no_cuda --mode score --resnet_shortcut A

loading model /content/drive/MyDrive/SkillWorkshop/resnet-34-kinetics-cpu.pth
/content/drive/MyDrive/SkillWorkshop/VideoProcess/SorryToBoderYou.mp4
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enab

In [8]:
! python generate_result_video/generate_result_video.py /content/drive/MyDrive/SkillWorkshop/output.json /content/drive/MyDrive/SkillWorkshop/VideoProcess /content/drive/MyDrive/SkillWorkshop/VideoPred class_names_list 5

/content/drive/MyDrive/SkillWorkshop/VideoProcess/SorryToBoderYou.mp4
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-

In [9]:
file = open('/content/drive/MyDrive/SkillWorkshop/output.json')
segments = json.load(file)

In [10]:
def Score(segment, clip):
  classNames = []
  with open('class_names_list', 'r') as class_names_list:
    index = 0
    for c in class_names_list:
      classNames.append({'index':index, 'name':c, 'score':0})
      index += 1
    for i in range(len(segments[segment]['clips'][clip]['scores'])):
      score = segments[segment]['clips'][clip]['scores'][i]
      classNames[i]['score'] = score
    
    classNames.sort(key = lambda x : x['score'], reverse = True)

    return classNames[0]

In [11]:
Score(0,5)

{'index': 224, 'name': 'playing chess\n', 'score': 5.94252872467041}

In [12]:
video = VideoFileClip(r"/content/drive/MyDrive/SkillWorkshop/VideoProcess/SorryToBoderYou.mp4")
subVideo = video.subclip(4,6)
subVideo.write_videofile("/content/drive/MyDrive/SkillWorkshop/VideoOut/SorryToBoderYou.mp4", audio_codec='aac')

Moviepy - Building video /content/drive/MyDrive/SkillWorkshop/VideoOut/SorryToBoderYou.mp4.
MoviePy - Writing audio in SorryToBoderYouTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/SkillWorkshop/VideoOut/SorryToBoderYou.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/SkillWorkshop/VideoOut/SorryToBoderYou.mp4


In [13]:
subVideo.ipython_display()

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [14]:
video.fps

23.976023976023978

In [15]:
frame_length = 1/video.fps

In [16]:
label_periods = []
for f in segments:
  print(f['video'])
  currentLabel = None
  ts = 0
  te = 0
  for c in f['clips']:
    label = c['label']
    s = c['segment'][0]
    e = c['segment'][1]
    if label == currentLabel:
      te = e 
    else:
      label_periods.append([(ts,te),currentLabel])

      ts = s
      te = e 
      currentLabel = label

SorryToBoderYou.mp4
TheBattleOfAlgiers.mp4
frame-030479.mp4
frame-028326.mp4
TheSixthSense.mp4


In [17]:
label_periods

[[(0, 0), None],
 [(1, 16), 'busking'],
 [(17, 48), 'blowing glass'],
 [(49, 64), 'busking'],
 [(65, 80), 'waiting in line'],
 [(81, 96), 'playing chess'],
 [(97, 160), 'marching'],
 [(161, 176), 'busking'],
 [(177, 208), 'waiting in line'],
 [(209, 224), 'giving or receiving award'],
 [(225, 240), 'kissing'],
 [(241, 272), 'smoking hookah'],
 [(273, 288), 'shaving head'],
 [(289, 320), 'applauding'],
 [(321, 336), 'air drumming'],
 [(337, 352), 'clapping'],
 [(353, 368), 'eating hotdog'],
 [(369, 384), 'crying'],
 [(385, 400), 'arm wrestling'],
 [(401, 416), 'smoking hookah'],
 [(417, 432), 'tasting beer'],
 [(433, 448), 'eating hotdog'],
 [(449, 464), 'smoking hookah'],
 [(465, 480), 'playing accordion'],
 [(481, 496), 'tasting beer'],
 [(497, 512), 'drinking beer'],
 [(513, 528), 'pumping fist'],
 [(529, 544), 'playing recorder'],
 [(0, 0), None],
 [(1, 16), 'parkour'],
 [(17, 32), 'marching'],
 [(33, 64), 'playing trombone'],
 [(65, 80), 'unloading truck'],
 [(81, 96), 'marching'],

In [18]:
len(label_periods)

129

In [19]:
longPeriods = []
for lp in label_periods:
  if lp[0][1]-lp[0][0] > 2/frame_length:
    longPeriods.append(lp)

In [20]:
longPeriods

[[(97, 160), 'marching'],
 [(129, 192), 'skateboarding'],
 [(225, 288), 'skateboarding'],
 [(337, 464), 'marching'],
 [(561, 624), 'air drumming'],
 [(401, 512), 'archery'],
 [(65, 128), 'kissing'],
 [(129, 208), 'trimming or shaving beard'],
 [(225, 304), 'trimming or shaving beard'],
 [(353, 416), 'reading newspaper']]

In [21]:
clips = []
for l in longPeriods:
  subclip = video.subclip(l[0][0]*frame_length,l[0][1]*frame_length)
  clips.append([subclip,l[1]+'_'+str(int(l[0][0]*frame_length))])

In [22]:
clips

[[<moviepy.video.io.VideoFileClip.VideoFileClip at 0x7f05ee60c3a0>,
  'marching_4'],
  'skateboarding_5'],
  'skateboarding_9'],
  'marching_14'],
  'air drumming_23'],
  'archery_16'],
  'kissing_2'],
  'trimming or shaving beard_5'],
  'trimming or shaving beard_9'],
  'reading newspaper_14']]

In [23]:
clips[0][0].ipython_display()

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [24]:
folderPath = '/content/drive/MyDrive/SkillWorkshop/Output/'
for c in clips:
  c[0].write_videofile(folderPath + c[1] + '.mp4', audio_codec='aac')

Moviepy - Building video /content/drive/MyDrive/SkillWorkshop/Output/marching_4.mp4.
MoviePy - Writing audio in marching_4TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/SkillWorkshop/Output/marching_4.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/SkillWorkshop/Output/marching_4.mp4
Moviepy - Building video /content/drive/MyDrive/SkillWorkshop/Output/skateboarding_5.mp4.
MoviePy - Writing audio in skateboarding_5TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/SkillWorkshop/Output/skateboarding_5.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/SkillWorkshop/Output/skateboarding_5.mp4
Moviepy - Building video /content/drive/MyDrive/SkillWorkshop/Output/skateboarding_9.mp4.
MoviePy - Writing audio in skateboarding_9TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/SkillWorkshop/Output/skateboarding_9.mp4
Moviepy - Building video /content/drive/MyDrive/SkillWorkshop/Output/marching_14.mp4.
MoviePy - Writing audio in marching_14TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/SkillWorkshop/Output/marching_14.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/SkillWorkshop/Output/marching_14.mp4
Moviepy - Building video /content/drive/MyDrive/SkillWorkshop/Output/air drumming_23.mp4.
MoviePy - Writing audio in air drumming_23TEMP_MPY_wvf_snd.mp4


chunk:   0%|          | 0/58 [00:00<?, ?it/s, now=None]

OSError: ignored